In [2]:
import requests
import re
import json
from tqdm import tqdm

# Metacritic API url
url = "https://mcgqlapi.com/api"

# Using readlines()
albums_file = open('albums.txt', 'r')
albums_lines = albums_file.readlines()

albums = {}

# Obtain information using API
for album_line in tqdm(albums_lines, desc='Querying MetacriticAPI...', total=len(albums_lines)):
    try:
        title, artist, genre, year, label, sales = re.findall('(.*) - (.*) \((.*)\) - (\d{4}) - (.*) - (.*)', album_line)[0]

        query = (
          'query {',
          '  album(input: {',
          f'    title: "{title}",',
          f'    artist: "{artist}",',
          f'    genre: "{genre}",',
          f'    year: "{year}",',
          f'    label: "{label}",',
          f'    sales: "{sales}"',
          '  }) {',
              '  title',
              '  artist',
              '  criticScore',
              '  userScore',
              '  year',
              '  genres',
              '  label',
              '  numOfCriticReviews',
              '  numOfUserReviews',
          '  }',
          '}'
        )

        query = '\n'.join(query)

        response = requests.post(url=url, json={"query": query})
        if response.status_code == 200:
            albums.append(response.json())
    except IndexError:
        # Skip malformed line
        continue

with open('albums.json', 'w') as output_file:
    output_file.write(json.dumps(albums))



Querying MetacriticAPI...: 100%|██████████████| 125/125 [00:53<00:00,  2.33it/s]


In [3]:
import json
import pandas as pd
import csv


# Abra o arquivo JSON e carregue o conteúdo em uma variável
with open('albums.json', 'r') as file:
    data = json.load(file)

with open('albums.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'artist', 'genre', 'year', 'label', 'sales'])
    
    for album_title, album_data in data.items():
        album = album_data['data']
        if((album.get('album'))):
            writer.writerow([
                album['album']['title'],
                album['album']['artist'],
                album['album']['genre'],
                album['album']['year'],
                album['album']['label'],
                album['album']['sales']
            ])

In [4]:
pd.read_csv('albums.csv')

,title,artist,genre,year,label,sales
